## Hash Sets and Hash Maps

Hash sets and maps are data structure commonly used in programming...,

In R, you can use them via package r2r:

In [1]:
install.packages('r2r')
x <- c("Hello", "World")
print(x)
2+2

Installing package into ‘/home/jakub/R/x86_64-pc-linux-gnu-library/4.4’
(as ‘lib’ is unspecified)



[1] "Hello" "World"


[1] 4

To illustrate the principle:

In [2]:
small_list = list(c(1:20))
for(x in small_list){
    print(x)    
}

 [1]  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20
